# Домашнее задание № 3. Исправление опечаток

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [3]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [5]:
import textdistance
from difflib import get_close_matches
from tqdm import tqdm

In [6]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [7]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Alex\AppData\Local\Temp\ipykernel_29848\2167344717.py:1: SyntaxWarning: invalid escape sequence '\w'
  vocab = Counter(re.findall('\w+', corpus.lower()))


In [8]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
        
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    for i in range(1, len(closest)-1):
        word = closest[i][0]
        s_word = closest[i][1]
        prev_word = closest[i-1][0]
        s_prev_word = closest[i-1][1]
        if s_word == s_prev_word:
            p_word = P(word)
            p_prev = P(prev_word)
            if p_word>p_prev:
                closest.pop(i-1)
            else:
                closest.pop(i)
                
    return (closest[0])

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]


In [9]:
get_closest_hybrid_match('сонце', X, vec)

('солнце', 0.8333333333333334)

In [10]:
get_closest_hybrid_match('лесница', X, vec)

('лестница', 0.875)

In [11]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1],get_closest_hybrid_match(pair[1], X, vec)[0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████| 915/915 [08:55<00:00,  1.71it/s]


In [12]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8552276138069035
0.48524844720496896
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [13]:
def del_char(word):
    char_list=[]
    del_word = str()
    del_list=[word]
    for char in word:
        char_list.append(char)
    n= len(char_list)
    for i in range(n):
        new_chars = list(char_list)
        new_chars.pop(i)
        del_word = ''.join(j for j in new_chars)
        del_list.append(del_word)
    return del_list 

#cоставляется словарь правильных слов

good_words = set(w for w in vocab)

#cоставляется словарь удалений

del_dict = dict()
for word in good_words:
    del_words = del_char(word)
    for dword in del_words:
        if dword not in del_dict:
            del_dict[dword] = {word}
        else:
            del_dict[dword].add(word)

#генерируются все варианты удаления,  выбираются те, что есть в словаре удалений. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления

def check_Symspell(word):
    deleted = del_char(word)
    candids = []
    for dels in deleted:
        for i in del_dict:
            if dels == i:
                for n in del_dict[i]:
                    candids.append(n)
    probs_c = dict()
    if len(candids)>1:
        for c in candids:
            c_prob = P(c)
            probs_c [c] = c_prob
            probs_dict = {k: v for k, v in sorted(probs_c.items(), key=lambda item: item[1], reverse=True)}
        res = list(probs_dict.keys())[0]
    else:
        res = candids
    return res

In [14]:
check_Symspell('сабака')

'собака'

In [15]:
check_Symspell('гаворить')

['говорить']

In [16]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1],check_Symspell(pair[1]))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████| 915/915 [15:45<00:00,  1.03s/it]


In [17]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8212106053026513
0.22127329192546583
0.09004249454461927
